I have had two simple raspberry pi weather stations running for a while now.

Both have pressure, temperature and humidity sensors.

One I have in the *carefully controlled environment* of my study, the
other is hanging out of the window.

The study one is known as *pijessie* as it started life as a Raspberry
Pi running the Jessie version of Raspbian.

The outside station is known as *kittycam* as I intend at some point
to attach a camera so I can watch our cat come and go.

For a while I have been noticing that the pressure values have been
quite a way apart.  The software I am includes a conversion to
altitude and I find these numbers more natural for me to think about.

The altitude conversion assumes the pressure at sea level is 1023.25
hPa, which is the mean pressure at sea level.

When the pressure is higher than this the altitude comes out below sea
level, when pressure is lower than this above sea level.

As always, Wikipedia has good information on this:
https://en.wikipedia.org/wiki/Atmospheric_pressure

For a while I had been noticing the two sensors giving values
differing by about 10 metres altitude.

I had put this down to the sensors not being calibrated accurately,
but also noticed that kittycam was more prone to weird glitches.

Now the glitches I put down to the fact I have one process collecting
data every minute and another process creating a display on my laptop
so I can glance over and see what the weather is doing.  The latter
was just polling the sensor every 10 minutes.

The code does not do anything smart like get a lock and my guess was
that the two processes were occasionally trampling on each other's feet.

Long story short, I decided to take a closer look.

In [1]:
# Tell matplotlib to plot in line
%matplotlib inline

import datetime

# import pandas
import pandas

# seaborn magically adds a layer of goodness on top of Matplotlib
# mostly this is just changing matplotlib defaults, but it does also
# provide some higher level plotting methods.
import seaborn

# Tell seaborn to set things up
seaborn.set()


In [2]:
# input files: the data from the two sensors
infiles = ["../files/kittycam_weather.csv", "../files/pijessie_weather.csv"]


In [3]:
# Read the data

data = []

for infile in infiles:
    data.append(pandas.read_csv(infile, index_col='date', parse_dates=['date']))


In [4]:
# take a look at what we got
data[0].describe()

               temp       pressure      altitude  sealevel_pressure  \
count  42768.000000   42768.000000  42768.000000       42768.000000   
mean      28.114357  101385.375842     -5.068642      101387.094487   
std        2.205370     430.276252     36.334073         409.738250   
min       22.800000   56117.000000  -1314.018026       67136.000000   
25%       26.500000  101141.750000    -26.939647      101142.000000   
50%       27.700000  101411.000000     -7.157439      101412.000000   
75%       29.900000  101648.000000     15.246750      101649.000000   
max       43.200000  102242.000000   3397.334521      118353.000000   

           humidity      temp_dht  
count  42764.000000  42764.000000  
mean      76.806599     27.730374  
std        8.512122      2.068583  
min       29.500000     22.299999  
25%       71.599998     26.200001  
50%       77.099998     27.400000  
75%       83.500000     29.299999  
max       94.300003     47.099998  

In [5]:
# plots are always good 

data[0].plot(subplots=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fedb5fa2668>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7fedb64ffe10>], dtype=object)

Now the two sets of data have different indices since the processes
collecting the data are not in sync.

So we need to align the data and then fill in missing values

In [6]:
# align returns two new dataframes, now aligned
d1, d2 = data[0].align(data[1])

In [7]:
# have a look, note the count is just the valid data.
# Things have been aligned, but missing values are set ton NaN
d1.describe()

               temp       pressure      altitude  sealevel_pressure  \
count  42768.000000   42768.000000  42768.000000       42768.000000   
mean      28.114357  101385.375842     -5.068642      101387.094487   
std        2.205370     430.276252     36.334073         409.738250   
min       22.800000   56117.000000  -1314.018026       67136.000000   
25%       26.500000  101141.750000    -26.939647      101142.000000   
50%       27.700000  101411.000000     -7.157439      101412.000000   
75%       29.900000  101648.000000     15.246750      101649.000000   
max       43.200000  102242.000000   3397.334521      118353.000000   

           humidity      temp_dht  
count  42764.000000  42764.000000  
mean      76.806599     27.730374  
std        8.512122      2.068583  
min       29.500000     22.299999  
25%       71.599998     26.200001  
50%       77.099998     27.400000  
75%       83.500000     29.299999  
max       94.300003     47.099998  

In [8]:
# Use interpolation to fill in the missing values
d1 = d1.interpolate(method='time')
d2 = d2.interpolate(method='time')



In [9]:
# Now plot
d1.altitude.plot()
print(len(d1))

80476


In [10]:
# For convenience, add a new series to d1 with the altitude data from d2
d1['altitude2'] = d2.altitude

In [11]:
# Now plot the two
d1[['altitude', 'altitude2']][10000:30000].clip(-60,60).plot()

In [12]:
(d1.altitude - d1.altitude2)[10000:30000].clip(-20,15).plot()

So we do have a difference around 5m.  More interestingly, there seems to be some sort of daily pattern to the data.

## How the BMP180 sensor works

The BMP180 pressure sensor includes a temperature sensor.  It also
includes a number of constants stored in EPROM for calibration.

The raw reading from the pressure sensor gives different values if the
pressure stays constant but the temperature varies.

The data sheet
https://www.adafruit.com/datasheets/BST-BMP180-DS000-09.pdf for the
sensor describes the calculations that have to be made to compensate
for the temperature.

The code below, from the Adafruit code for the sensor implements these
calculations.

In [14]:
class BMP180(object):
    def read_pressure(self):
        """Gets the compensated pressure in Pascals."""
        UT = self.read_raw_temp()
        UP = self.read_raw_pressure()
        # Datasheet values for debugging:
        #UT = 27898
        #UP = 23843
        # Calculations below are taken straight from section 3.5 of the datasheet.
        # Calculate true temperature coefficient B5.
        X1 = ((UT - self.cal_AC6) * self.cal_AC5) >> 15
        X2 = (self.cal_MC << 11) / (X1 + self.cal_MD)
        B5 = X1 + X2

        # Pressure Calculations
        B6 = B5 - 4000

        X1 = (self.cal_B2 * (B6 * B6) >> 12) >> 11
        X2 = (self.cal_AC2 * B6) >> 11
        X3 = X1 + X2
        B3 = (((self.cal_AC1 * 4 + X3) << self._mode) + 2) / 4

        X1 = (self.cal_AC3 * B6) >> 13
        X2 = (self.cal_B1 * ((B6 * B6) >> 12)) >> 16
        X3 = ((X1 + X2) + 2) >> 2
        B4 = (self.cal_AC4 * (X3 + 32768)) >> 15

        B7 = (UP - B3) * (50000 >> self._mode)

        if B7 < 0x80000000:
            p = (B7 * 2) / B4
        else:
            p = (B7 / B4) * 2
        X1 = (p >> 8) * (p >> 8)
        X1 = (X1 * 3038) >> 16
        X2 = (-7357 * p) >> 16
        p = p + ((X1 + X2 + 3791) >> 4)

        return p

    

In [ ]:
cal = {
 'cal_AC1': 7544,
 'cal_AC2': -1141,
 'cal_AC3': -14468,
 'cal_AC4': 33846,
 'cal_AC5': 25593,
 'cal_AC6': 19416,
 'cal_B1': 6515,
 'cal_B2': 43,
 'cal_MB': -32768,
 'cal_MC': -11786,
 'cal_MD': 2459}


In [ ]:
dataseet_cal = {
 'cal_AC1': 408,
 'cal_AC2': -72,
 'cal_AC3': -14383,
 'cal_AC4': 32741,
 'cal_AC5': 32757,
 'cal_AC6': 23153,
 'cal_B1': 6190,
 'cal_B2': 4,
 'cal_MB': -32767,
 'cal_MC': -8711,
 'cal_MD': 2868}
